In [7]:
import pandas as pd
import numpy as np

# Load Porter dataset
porter = pd.read_csv("../data/delivery.csv")

# Load Olist datasets
orders = pd.read_csv("../data/olist_orders_dataset.csv")
items = pd.read_csv("../data/olist_order_items_dataset.csv")
customers = pd.read_csv("../data/olist_customers_dataset.csv")
geo = pd.read_csv("../data/olist_geolocation_dataset.csv")
sellers = pd.read_csv("../data/olist_sellers_dataset.csv")

# Print shapes
datasets = {
    "Porter": porter,
    "Orders": orders,
    "Items": items,
    "Customers": customers,
    "Geo": geo,
    "Sellers": sellers
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")

porter.head()


Porter: (175777, 14)
Orders: (99441, 8)
Items: (112650, 7)
Customers: (99441, 5)
Geo: (1000163, 5)
Sellers: (3095, 4)


,market_id,created_at,actual_delivery_time,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:11:17,4,1.0,4,3441,4,557,1239,33.0,14.0,21.0,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:33:25,46,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,690.0
2,2.0,2015-02-16 00:11:35,2015-02-16 01:06:35,36,3.0,4,4771,3,820,1604,8.0,6.0,18.0,289.0
3,1.0,2015-02-12 03:36:46,2015-02-12 04:35:46,38,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,795.0
4,1.0,2015-01-27 02:12:36,2015-01-27 02:58:36,38,1.0,2,3620,2,1425,2195,5.0,5.0,7.0,205.0


In [8]:
# --- Cell 2: Convert date columns & create delivery time metrics ---

# Convert Porter dataset date columns
porter['created_at'] = pd.to_datetime(porter['created_at'], errors='coerce')
porter['actual_delivery_time'] = pd.to_datetime(porter['actual_delivery_time'], errors='coerce')

# Calculate delivery duration in minutes
porter['delivery_time_minutes'] = (porter['actual_delivery_time'] - porter['created_at']).dt.total_seconds() / 60

# Convert Olist date columns
date_cols_orders = ['order_purchase_timestamp', 'order_approved_at',
                    'order_delivered_carrier_date', 'order_delivered_customer_date',
                    'order_estimated_delivery_date']

for col in date_cols_orders:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

print("✔ Date conversion complete.")
porter[['created_at', 'actual_delivery_time', 'delivery_time_minutes']].head()


✔ Date conversion complete.


,created_at,actual_delivery_time,delivery_time_minutes
0,2015-02-06 22:24:17,2015-02-06 23:11:17,47.0
1,2015-02-10 21:49:25,2015-02-10 22:33:25,44.0
2,2015-02-16 00:11:35,2015-02-16 01:06:35,55.0
3,2015-02-12 03:36:46,2015-02-12 04:35:46,59.0
4,2015-01-27 02:12:36,2015-01-27 02:58:36,46.0


In [9]:
# --- Cell 3: Handle missing values & clean invalid rows ---

print("Missing values before cleaning:")
print("\nPorter:\n", porter.isna().sum())
print("\nOrders:\n", orders.isna().sum())

# 1. Remove Porter rows where delivery time is negative or null
porter = porter.dropna(subset=['delivery_time_minutes'])
porter = porter[porter['delivery_time_minutes'] >= 0]

# 2. Remove Olist orders where purchase time is missing
orders = orders.dropna(subset=['order_purchase_timestamp'])

# 3. Fix invalid numeric values in Porter data
num_cols = ['total_items', 'subtotal', 'min_item_price', 'max_item_price']
for col in num_cols:
    porter = porter[porter[col] >= 0]   # remove negative values if present

# 4. Reset index after cleaning
porter.reset_index(drop=True, inplace=True)
orders.reset_index(drop=True, inplace=True)

print("\n✔ Cleaning complete.")
print("Porter shape after cleaning:", porter.shape)
print("Orders shape after cleaning:", orders.shape)

porter.head()


Missing values before cleaning:

Porter:
 market_id                                       0
created_at                                      0
actual_delivery_time                            0
store_primary_category                          0
order_protocol                                  0
total_items                                     0
subtotal                                        0
num_distinct_items                              0
min_item_price                                  0
max_item_price                                  0
total_onshift_dashers                           0
total_busy_dashers                              0
total_outstanding_orders                        0
estimated_store_to_consumer_driving_duration    0
delivery_time_minutes                           0
dtype: int64

Orders:
 order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                

,market_id,created_at,actual_delivery_time,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_store_to_consumer_driving_duration,delivery_time_minutes
0,1.0,2015-02-06 22:24:17,2015-02-06 23:11:17,4,1.0,4,3441,4,557,1239,33.0,14.0,21.0,861.0,47.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:33:25,46,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,690.0,44.0
2,2.0,2015-02-16 00:11:35,2015-02-16 01:06:35,36,3.0,4,4771,3,820,1604,8.0,6.0,18.0,289.0,55.0
3,1.0,2015-02-12 03:36:46,2015-02-12 04:35:46,38,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,795.0,59.0
4,1.0,2015-01-27 02:12:36,2015-01-27 02:58:36,38,1.0,2,3620,2,1425,2195,5.0,5.0,7.0,205.0,46.0


In [10]:
# --- Cell 4: Feature Engineering ---

# Extract time-based features from Porter dataset
porter['order_hour'] = porter['created_at'].dt.hour
porter['order_day'] = porter['created_at'].dt.day_name()
porter['order_month'] = porter['created_at'].dt.month
porter['order_week'] = porter['created_at'].dt.isocalendar().week.astype(int)

# Classify weekday vs weekend
porter['is_weekend'] = porter['order_day'].isin(['Saturday', 'Sunday']).astype(int)

# Identify rush hours (customizable)
rush_hours = [11, 12, 13, 18, 19, 20]
porter['is_rush_hour'] = porter['order_hour'].isin(rush_hours).astype(int)

# Bucketize delivery time
def bucket_delivery_time(x):
    if x <= 20: return "0-20 min"
    elif x <= 40: return "20-40 min"
    elif x <= 60: return "40-60 min"
    else: return "60+ min"

porter['delivery_bucket'] = porter['delivery_time_minutes'].apply(bucket_delivery_time)

# Item count buckets
def item_bucket(x):
    if x <= 2: return "1-2 items"
    elif x <= 5: return "3-5 items"
    else: return "6+ items"

porter['item_count_bucket'] = porter['total_items'].apply(item_bucket)

# Price category
porter['order_value_bucket'] = pd.cut(
    porter['subtotal'],
    bins=[0, 1000, 3000, 7000, porter['subtotal'].max()],
    labels=['Low', 'Medium', 'High', 'Very High']
)

print("✔ Feature Engineering Complete.")
porter[['created_at', 'order_hour', 'order_day', 'is_weekend', 'is_rush_hour', 'delivery_bucket']].head()


✔ Feature Engineering Complete.


,created_at,order_hour,order_day,is_weekend,is_rush_hour,delivery_bucket
0,2015-02-06 22:24:17,22,Friday,0,0,40-60 min
1,2015-02-10 21:49:25,21,Tuesday,0,0,40-60 min
2,2015-02-16 00:11:35,0,Monday,0,0,40-60 min
3,2015-02-12 03:36:46,3,Thursday,0,0,40-60 min
4,2015-01-27 02:12:36,2,Tuesday,0,0,40-60 min


In [11]:
# --- Cell 5: Merge Porter + Olist datasets to build a master table ---

# STEP 1: Standardize column names for easier merging
porter = porter.rename(columns={
    "created_at": "order_created_at",
    "actual_delivery_time": "order_delivered_at"
})

# Ensure common join key exists in both datasets
# Porter does NOT contain order_id → we will perform a "cross-merge" only on time features
# Instead, we will merge Olist datasets together to create a rich table

# STEP 2: Merge Orders + Customers
orders_customers = orders.merge(
    customers,
    on="customer_id",
    how="left"
)

# STEP 3: Merge Orders + Items (each order may have multiple items)
orders_items = orders.merge(
    items,
    on="order_id",
    how="left"
)

# STEP 4: Combine Orders + Items + Customers
olist_full = orders_items.merge(
    customers,
    on="customer_id",
    how="left"
)

print("✔ Olist dataset merged:", olist_full.shape)

# STEP 5: For demonstration, we align Porter + Olist on date (not order_id)
# Because Porter does not contain order ids, only timestamps.
# So we will create 'date' columns to join on daily patterns.

porter['order_date'] = porter['order_created_at'].dt.date
olist_full['order_date'] = olist_full['order_purchase_timestamp'].dt.date

# Merge on date
master_df = porter.merge(
    olist_full,
    on="order_date",
    how="left",
    suffixes=("_porter", "_olist")
)

print("✔ Master dataset created:", master_df.shape)

master_df.head()


✔ Olist dataset merged: (113425, 18)
✔ Master dataset created: (175765, 43)


,market_id,order_created_at,order_delivered_at,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,...,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,1.0,2015-02-06 22:24:17,2015-02-06 23:11:17,4,1.0,4,3441,4,557,1239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2015-02-10 21:49:25,2015-02-10 22:33:25,46,2.0,1,1900,1,1400,1400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2015-02-16 00:11:35,2015-02-16 01:06:35,36,3.0,4,4771,3,820,1604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2015-02-12 03:36:46,2015-02-12 04:35:46,38,1.0,1,1525,1,1525,1525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,2015-01-27 02:12:36,2015-01-27 02:58:36,38,1.0,2,3620,2,1425,2195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# --- Cell 5.1: Add geolocation (lat/lng) + compute distance_km ---

# Step 1: Prepare geo dataset (zipcode → lat/lng)
geo_clean = geo.groupby("geolocation_zip_code_prefix").agg({
    "geolocation_lat": "mean",
    "geolocation_lng": "mean"
}).reset_index()

geo_clean.columns = ["customer_zip_code_prefix", "customer_lat", "customer_lng"]

# Step 2: Merge geolocation into master_df (customer side)
master_df = master_df.merge(
    geo_clean,
    left_on="customer_zip_code_prefix",
    right_on="customer_zip_code_prefix",
    how="left"
)

# Step 3: For store location, assign dummy fixed store coordinates (Porter dataset has no store lat/lng)
# Example: Assume store is located in Bangalore (12.9716, 77.5946)
store_lat, store_lng = 12.9716, 77.5946

master_df['store_lat'] = store_lat
master_df['store_lng'] = store_lng

# Step 4: Define Haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # KM
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    a = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Step 5: Compute distance
master_df['distance_km'] = haversine(
    master_df['customer_lat'],
    master_df['customer_lng'],
    master_df['store_lat'],
    master_df['store_lng']
)

print("✔ distance_km added")
master_df[['customer_lat', 'customer_lng', 'store_lat', 'store_lng', 'distance_km']].head()


✔ distance_km added


,customer_lat,customer_lng,store_lat,store_lng,distance_km
0,NaN,NaN,12.9716,77.5946,NaN
1,NaN,NaN,12.9716,77.5946,NaN
2,NaN,NaN,12.9716,77.5946,NaN
3,NaN,NaN,12.9716,77.5946,NaN
4,NaN,NaN,12.9716,77.5946,NaN


In [13]:
# --- Cell 6: Final cleanup & save clean datasets ---

# STEP 1: Remove duplicate or useless columns (like repeated city_x/city_y etc.)
master_df = master_df.loc[:, ~master_df.columns.duplicated()]

# STEP 2: Keep only useful final columns
final_cols = [
    # Porter fields
    'order_created_at', 'order_delivered_at', 'order_date',
    'delivery_time_minutes', 'total_items', 'subtotal',
    'min_item_price', 'max_item_price',
    'order_hour', 'order_day', 'is_weekend', 'is_rush_hour',
    'delivery_bucket', 'item_count_bucket', 'order_value_bucket',
    
    # Courier load
    'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders',
    
    # Olist order fields
    'order_id', 'customer_id', 'seller_id',
    'price', 'freight_value',
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    
    # Customer fields
    'customer_city', 'customer_state',
]

# Only keep columns that actually exist (avoids key errors)
final_cols = [col for col in final_cols if col in master_df.columns]

master_clean = master_df[final_cols].copy()

# STEP 3: Save cleaned datasets
import os
os.makedirs("../dashboard/data_cleaned", exist_ok=True)

master_clean.to_csv("../dashboard/data_cleaned/master_clean.csv", index=False)
porter.to_csv("../dashboard/data_cleaned/porter_clean.csv", index=False)
olist_full.to_csv("../dashboard/data_cleaned/olist_full_clean.csv", index=False)

print("✔ Cleaning Complete & Files Saved")
print("Saved files:")
print("- master_clean.csv")
print("- porter_clean.csv")
print("- olist_full_clean.csv")

master_clean.head()


✔ Cleaning Complete & Files Saved
Saved files:
- master_clean.csv
- porter_clean.csv
- olist_full_clean.csv


,order_created_at,order_delivered_at,order_date,delivery_time_minutes,total_items,subtotal,min_item_price,max_item_price,order_hour,order_day,...,customer_id,seller_id,price,freight_value,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,customer_city,customer_state
0,2015-02-06 22:24:17,2015-02-06 23:11:17,2015-02-06,47.0,4,3441,557,1239,22,Friday,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
1,2015-02-10 21:49:25,2015-02-10 22:33:25,2015-02-10,44.0,1,1900,1400,1400,21,Tuesday,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
2,2015-02-16 00:11:35,2015-02-16 01:06:35,2015-02-16,55.0,4,4771,820,1604,0,Monday,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
3,2015-02-12 03:36:46,2015-02-12 04:35:46,2015-02-12,59.0,1,1525,1525,1525,3,Thursday,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
4,2015-01-27 02:12:36,2015-01-27 02:58:36,2015-01-27,46.0,2,3620,1425,2195,2,Tuesday,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
